# Question 1

In [1]:
# Write a code for printing factorial of a number using recursion

def factorial(N):
    '''
    A factorial of N is defined as 

    N * (N-1) * (N-2) * (N-3) * (N-4) ... 1 

    example:
        factorial of 4 is = `4*3*2*1 = 24` 
    '''
    if N==1:
        return 1 
    else:
        return N*factorial(N-1)
    
factorial(3)

6

# Question 2


In [2]:
# Write a lambda code for checking if a number if even or odd and then use it to filter odd numbers from a given list

checker = lambda x : True if x%2 ==1 else False
s=range(0,100,1)
filtered_list = list(filter(checker,s))
filtered_list

[1,
 3,
 5,
 7,
 9,
 11,
 13,
 15,
 17,
 19,
 21,
 23,
 25,
 27,
 29,
 31,
 33,
 35,
 37,
 39,
 41,
 43,
 45,
 47,
 49,
 51,
 53,
 55,
 57,
 59,
 61,
 63,
 65,
 67,
 69,
 71,
 73,
 75,
 77,
 79,
 81,
 83,
 85,
 87,
 89,
 91,
 93,
 95,
 97,
 99]

# Question 3 

In [3]:
# Create a file containing factors of number 5000, make sure each factor is written in a different line

N=5000
factors = ''

for x in range(1,(N//2) +1):
    if N%x==0:
        factors=factors+str(x)+"\n"

factors = factors + str(N)
with open('factorsOf5000.txt','w') as f:
    f.write(factors)

# Question 4

In [4]:
# Some additional git hub public apis 

import requests
username = input("Enter the github username:")
request = requests.get('https://api.github.com/users/'+username+'/starred')
json = request.json()
for i in range(0,len(json)):
#     print(json[i])
    print("Project Number:",i+1)
    print("Project Name:",json[i]['name'])
    print("Project URL:",json[i]['svn_url'],"\n")

Enter the github username:akashgandhi51
Project Number: 1
Project Name: Outlier-Detection
Project URL: https://github.com/akashgandhi51/Outlier-Detection 

Project Number: 2
Project Name: Missing-Data-Imputation
Project URL: https://github.com/akashgandhi51/Missing-Data-Imputation 



# Question 5

In [5]:
# Create a code which Queries reddit/twitter html pages and store the outputs using multi threaded code
import requests
import uuid
from concurrent.futures import ThreadPoolExecutor, as_completed

pagelist=['https://twitter.com/TechNation','https://twitter.com/Apple','https://twitter.com/MKBHD','https://www.reddit.com/r/artificial/','https://www.reddit.com/r/Volvo/','https://www.reddit.com/r/technology/']


def filedownloader(url,filename):
    try:
        html=requests.get(url,stream=True)
        open(f'{filename}.txt','wb').write(html.content)
        return html.status_code
    except requests.exceptions.RequestException as e:
        return e

def thread():
    threads= []
    with ThreadPoolExecutor(max_workers=5) as executor:
        for url in pagelist:
            file_name = uuid.uuid1()
            threads.append(executor.submit(filedownloader, url, file_name))

        for task in as_completed(threads):
            print(task.result()) 
    
with ThreadPoolExecutor(max_workers=5) as executor:
    thread()

200
200
200
200
200
200


In [ ]:
### Alternate Way to get data using Reddit APIs

import praw
import pandas as pd

# Acessing the reddit api
reddit = praw.Reddit(client_id="",#my client id
                     client_secret="",  #your client secret
                     user_agent="my user agent", #user agent name
                     username = "",     # your reddit username
                     password = "")     # your reddit password
### Note: How to get the above values: 
# https://www.geeksforgeeks.org/how-to-get-client_id-and-client_secret-for-python-reddit-api-registration/

sub = ['Askreddit']  # make a list of subreddits you want to scrape the data from

for s in sub:
    subred = reddit.subreddit(s)   # Chosing the subreddit

########################################
#   CREATING DICTIONARY TO STORE THE DATA WHICH WILL BE CONVERTED TO A DATAFRAME
########################################

#   NOTE: ALL THE POST DATA AND COMMENT DATA WILL BE SAVED IN TWO DIFFERENT
#   DATASETS AND LATER CAN BE MAPPED USING IDS OF POSTS/COMMENTS AS WE WILL 
#   BE CAPTURING ALL IDS THAT COME IN OUR WAY

# SCRAPING CAN BE DONE VIA VARIOUS STRATEGIES {HOT,TOP,etc} we will go with keyword strategy i.e using search a keyword
    query = ['DataScience']

    for item in query:
        post_dict = {
            "title" : [],
            "score" : [],
            "id" : [],
            "url" : [],
            "comms_num": [],
            "created" : [],
            "body" : []
        }
        comments_dict = {
            "comment_id" : [],
            "comment_parent_id" : [],
            "comment_body" : [],
            "comment_link_id" : []
        }
        for submission in subred.search(query,sort = "top",limit = 1):
            post_dict["title"].append(submission.title)
            post_dict["score"].append(submission.score)
            post_dict["id"].append(submission.id)
            post_dict["url"].append(submission.url)
            post_dict["comms_num"].append(submission.num_comments)
            post_dict["created"].append(submission.created)
            post_dict["body"].append(submission.selftext)
            
            ##### Acessing comments on the post
            submission.comments.replace_more(limit = 1)
            for comment in submission.comments.list():
                comments_dict["comment_id"].append(comment.id)
                comments_dict["comment_parent_id"].append(comment.parent_id)
                comments_dict["comment_body"].append(comment.body)
                comments_dict["comment_link_id"].append(comment.link_id)
        
        post_comments = pd.DataFrame(comments_dict)

        post_comments.to_csv(s+"_comments_"+ item +"subreddit.csv")
        post_data = pd.DataFrame(post_dict)
        post_data.to_csv(s+"_"+ item +"subreddit.csv")